# Logistic Regression with Ship Data
**Before you begin.**
You will edit this Colab notebook by adding/changing code and adding text to answer questions and provide explanations and insights.  When you are finished, you can print to PDF in order to easily generate a report.  To earn full points,  
*   be sure that your code does not extend past the vertical bar in code cells. Use the double-backslash operator for multi-line code, if necessary,
*   print text along with the answer/explanation, if answering/explaining with code.  For example, use `print(f'The bias is {bias}.')` instead of `print(bias)`.  This way, your instructor knows that you know how to interpret what you are seeing,
*   use italicized text if answering/explaining with text.  This will help your answers stand out, and
*   identify and explain some key observations that were not explcitily asked for in this assignment.  Sometimes the instructor wants to see if you know what to look for on your own.

You can add cells of either text or code by hovering over the horizontal edge of the cell just before where you want to add a new cell.  Click either "+ text" or "+ code", as appopriate.  You may have to click in the previous cell in order to see "+ text" or "+ code"



---






**Assignment summary.** This assignment is an introduction to the [San Francisco ship aerial imagery dataset](https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery) and logistic regression.  You will use black-box functions to program a computer vision algorithm that learns to detect ships in the aerial imagery.  This is a two-class classificaiton problem: y = 1 indicating a ship is present in the image, and y = 0 for either no ship or partial (cropped) ship.  You will investigate model parameters, cross-entropy, and probabilities.

The images have been converted to black and white for ease of processing.  This black-and-white dataset will be used for many of the assignments you will complete throughout the remainder of the course.


---



**Get to know the dataset.** Connect this Colab notebook to your Google Drive.  This code is complete.

In [ ]:
# Given your permission, this will connect your notebook to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Load in the ship data from Google Drive and print out the number of images.  The name of the folder containing all the ships should be `shipdata_MLcourse`, otherwise you need to change the code below accordingly.  The variable `img_data` contains all of the pixel information for each of the images.  This code is complete.

In [ ]:
# Now the ship data can be loaded into the notebook (presuming you uploaded..
# ...the data previously into your Google Drive)
import os, pickle

my_path = '/content/drive/My Drive/shipdata_MLcourse'
with open(os.path.join(my_path, 'img_data.pickle'), 'rb') as handle:
  img_data = pickle.load(handle)

print(f'The number of images is {len(img_data)}.') # print the number of images
print(f'Each image is a vector of length {len(img_data[0])}.')
print(f'The shape of img_data is {img_data.shape}.')          

Get the targets vector `y`: 1 if the image contains a ship, 0 for no ship (or partial ship). This code is complete.


In [ ]:
with open(os.path.join(my_path, 'targets.pickle'), 'rb') as handle:
  y = pickle.load(handle)
  print(f'The length of y is {len(y)}.')

Check out some of the ship images!  Each image is a vector of length 6400 that can be reshaped into 80 by 80 to make a small picture.  Change the value of `my_index` to view different images.  Remember that Python indexing starts at zero.  The code will print out the target as well.  Images towards the end of the dataset (larger indices) contain the ships. The code is complete.

In [ ]:
from google.colab.patches import cv2_imshow

my_index = 1000        # change this number
img = img_data[my_index].reshape((80,80))
cv2_imshow(img)  # display image

if y[my_index] == 0:
  print('This image is no ship or partial ship.')
else:
  print('This image is ship.')

**Work with a dimensionally reduced dataset.** Each image is 80 x 80 = 6400 pixels, which, for reasons you will understand later in the course, is too large for logistic regression and other classification algorithms.  Instead you will work with vectors of length 300 that well describe the images.  We call this a "dimesionally reduced" dataset, since each image is now encoded with 300 numbers instead of 6400 numbers.  How exactly these 300-length vectors come to be is from a technique you will learn later in the course (called principal component analysis).

Load in the dimensionally reduced dataset.  The code below is complete.

In [ ]:
with open(os.path.join(my_path, 'X_pca.pickle'), 'rb') as handle:
  X = pickle.load(handle)
  print(f'The shape of X is {X.shape}.')

Employ the black-box function [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to train a predictive model.  The data are first normalized to zero mean and unit variance using the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) function, resulting in variable `Xnorm`.  You need to instantiate and fit the logistic regression model on `Xnorm` and `y`.  Then, use the trained model to predict the targets given `Xnorm`.  Your predictions should be stored as variable `yhat`.  This block of code may take around one minute to run.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xnorm = scaler.fit_transform(X)
# TODO: instantiate, fit, and predict using LogisticRegression, Xnorm, y

yhat = 

Count and print the number of correct predictions.  How did the model do?  The code below is complete but assumes your predictions from the previous block are stored as `yhat`, and is an array of shape (4000,).

In [ ]:
import numpy as np
num_correct = np.sum(y == yhat)
print(f'The model made {num_correct}/{len(y)} correct predictions.')

**Go deeper into the model.** Use the model's `predict_proba` method to investigate the decision probabilities.  What is the probability that the first image in the dataset contains a (full) ship?  What is the probability that the last image in the dataset contains a (full) ship?  What is the probability that an image you displayed above with `my_index` contains a (full) ship?  Can you find an image for which the probability of the assigned class is less than 0.75?  Display the image.  What was it predicted as?  Explain.

In [ ]:
# TODO: Write code to answer the questions above....
#  ...Use the predict_proba() method








Recall that a logistic regression model assigns weights to each feature, and adds the weighted features together with a bias to input into a sigmoid function.  If the $i$th image is represented as a 300-length vector (with bias) 
$$
\mathbf{x}_i = [1,~x_1(i),~x_2(i),~ \cdots,~x_{300}(i)],
$$
and the learned weights are 
$$
\mathbf{\theta} = [\theta_0,~\theta_1,~\theta_2~\cdots,~\theta_{300}]^T,
$$
then the probability of $\mathbf{x}_i$ belonging to class 1 (ship) is given by
$$
h_{\mathbf{\theta}}(\mathbf{x}_i) = \frac{1}{1+e^{-\mathbf{x}_i\mathbf{\theta}}}.
$$
Using the model's attributes `coef_` and `intercept_`, write code below to determine the learned weights and bias.  What is the weight corresponding to the last feature $\theta_{300}$?  The first feature $\theta_1$?

In [ ]:
# TODO: Write code to answer the questions above...  
# ...Use the model's coef_ and intercept_ attributes.







Recall that the goal of the logistic regression model is to minimize the cross-entropy function, also called log-loss:
$$
J(\mathbf{\theta}) = -\sum_{i}\big(y_i\log(h_{\mathbf{\theta}}(\mathbf{x}_i)) + (1-y_i)\log(1-h_{\mathbf{\theta}}(\mathbf{x}_i))\big),
$$
where $y_i$ is the target of $i$th example.  Use the [log_loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) function along with the actual and predicted targets `y` and `yhat`, respectively, to calculate the cross-entropy of the learned model.  To compare the result, the code computes a random vector of 0's and 1's to simulate a random guessing of ship/no-ship.  Also calculate the cross-entropy of this random vector with the actual targets.  Comment on your answers.

In [ ]:
from sklearn.metrics import log_loss
# TODO: print the cross-entropy of y with yhat.  Use log_loss.


yrandom = np.random.binomial(size=len(y), n=1, p= 0.5)
# TOD0: print the cross-entropy of y with yrandom  Use log_loss.


This is the end of what is required.  You can easily generate a report by navigating to File -> Print and then selecting to save to PDF.  You will turn in this PDF.  Be sure that the report shows your code's output (that is, you've run each cell), and your notes/explanations are italicized.  Continue on for extra insights.

**Go further.** Upon instantiating the model with `LogisticRegression`, the maximum number of iterations `max_iter` for the gradient descent routine may be changed.  Consult the online documentation to learn its default parameter.  What is the default parameter?  What happens if you use one-tenth of the default parameter to train the model?

In [ ]:
# TODO: instantiate and fit using LogisticRegression, Xnorm, y.





Recall that we had to use a dimensionally reduced dataset instead of the original 6400 pixels.  What happens if you train the model on the original datset `img_data` (with the default value of `max_iter`)?

In [ ]:
# TODO: instantiate and fit using LogisticRegression, img_data, y.


